In [1]:
import os
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader

from torchvision import datasets, models, transforms

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=False
set_seed(42)

In [4]:
from source import cifar_dataloader
path = './../data/'
loader_dict = cifar_dataloader.get_cifar100_dataloader(path = path, batch_size=128)
train_loader = loader_dict['train']
val_loader = loader_dict['val']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = models.resnet18(pretrained=False)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
epochs = 500

model = model.to(device)

# supervised-contrastive learning
- 사전학습

In [6]:
loader_dict = cifar_dataloader.get_cifar100_dataloader(path = path, batch_size=256)
train_loader = loader_dict['train']
val_loader = loader_dict['val']

Files already downloaded and verified
Files already downloaded and verified


In [7]:
x,y = next(iter(train_loader))

In [16]:
encoder = nn.Sequential(model.conv1, model.bn1, model.relu, model.maxpool,
                     model.layer1, model.layer2, model.layer3, model.layer4, model.avgpool)

classifier = nn.Sequential(nn.Flatten(), model.fc)

In [14]:
xx = encoder(x[:32].to(device))
classifier(xx).shape

torch.Size([32, 1000])

In [ ]:
# constrastive learning phase
# https://github.com/HobbitLong/SupContrast/blob/master/losses.py
